# Projeto 1 - Ciência dos Dados

Nome: Dom Ruan Suzano

Nome: Hudson Monteiro

## Contextualização
___

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

Carregando algumas bibliotecas para limpeza

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
!pip install emoji
import emoji

[nltk_data] Downloading package stopwords to C:\Users\Dom
[nltk_data]     Ruan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Confirmação do diretório

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Dom Ruan\2° Semestre Eng\C.Dados\Projeto_1_CDados


## Análise do Banco de Dados

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'pepsi.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head(10)

,Treinamento,Relevância
0,namoral n tem refrigerante melhor q o da pepsi...,1
1,a pepsi fez o logo baseado na gravidade sobre ...,1
2,"@erongranth neo química, kalunga e pepsi acho ...",1
3,mataria por um refri geladinho agr mas so tem ...,1
4,"@pitta_ai e dionísio, o cara viciado em pepsi",1
5,@jujuubuenoo pepsi® geladinha tem um gostinho ...,0
6,escreva isso ou aquilo usando a tag #sonkezsen...,0
7,"@elitavres pepsi® é tudo de bom, fala aí! 😉💙❤",0
8,@pepsibr pq as pepsi ta vindo tao sem gas????,1
9,ele só queria uma pepsi,1


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,"@trishteza uma pepsi® é coisa de outro mundo, ...",0
1,@cultwolverine pepsi arena ia ficar muito mais...,1
2,@alisonbap pepsi e melhor,1
3,realizando uma pesquisa científica com meus am...,0
4,pizza tomando minha pepsi com limão melhorou o...,1


___
## Classificador automático de sentimento



Nosso produto é o refrigerante pepsi,um refrigerante norte-americano com sabor de cola, consideramos como relevante os tweets que falavam bem ou mal tanto da marca como do produto, além disso exceções  como tweets feito pelo twitter da pepsi foram considerados irrelevantes mesmo que falassem bem.

Definindo funções

In [7]:
def stop_words(lista):
    nova_lista = []
    for a in lista:
        if a not in stopwords.words('portuguese'):
            nova_lista.append(a)
    return nova_lista

In [8]:
# Limpa os tweets, tira os pontos desnecessários
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[®,!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [9]:
def separa_emoji(text):
    modified=' '.join(emoji.get_emoji_regexp().split(text))
    return modified

In [10]:
# retira o arroba(@)
def limpa_arrobas(texto):
    novo_texto = []
    for t in texto.split():
        if t[0] != '@':
            novo_texto.append(t)
    novo_texto = ' '.join(novo_texto)
    return novo_texto

In [11]:
# Limpa os links do tweet
def limpa_links(texto):
    novo_texto = []
    for t in texto.split():
        if len(t) > 4 and t[0:4] != 'http':
            novo_texto.append(t)
    novo_texto = ' '.join(novo_texto)
    return novo_texto

In [12]:
#Função que deixa todas as letras de uma frase minuscúlas e separa ela em uma lista
def tokenization(string):
    return string.lower().split()

In [13]:
def limpa_texto(texto):
    texto = limpa_arrobas(limpa_links(separa_emoji(cleanup(texto))))
    return texto

In [14]:
# aplicando num texto
for a in range(300):
    texto = train.loc[a,"Treinamento"]
    texto = str(texto)
    texto = limpa_texto(texto)
    train.loc[a,"Treinamento"] = texto

In [15]:
for a in range(200):
    texto = test.loc[a,"Teste"]
    texto = str(texto)
    texto = limpa_texto(texto)
    test.loc[a,"Teste"] = texto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [16]:
# Transformar palavras em variáveis categóricas


In [17]:
# Aplicando as funções de limpeza


## Separando os tweets

Relevantes

frequências Absolutas

In [68]:
# Relevantes 
rel = train[train['Relevância'] ==1]

# frequencia absoluta
rel_freq_abs = rel.value_counts()
rel_freq_abs

Treinamento                                                                        Relevância  Modelo
pepsi                                                                              1           1         9
pepsi melhor                                                                       1           1         4
única coisa compensou pepsi infelizmente daquela garrafinha                        1           1         1
gosto pepsi pepsi twist outro nível otimo refrigerante                             1           1         1
gostar pepsi negar décadas fazem campanhas marketing serão lembradas sempre delas  1           1         1
                                                                                                        ..
pepsi melhor passagem                                                              1           1         1
pepsi melhor dependente tratar                                                     1           1         1
pepsi maracujá                            

Irrelevantes

frequências Absolutas

In [52]:
# Irrelevantes
irr = train[train['Relevância'] ==0]

# Frequancia absoluta
irr_freq_abs = irr.value_counts()

irr_freq_abs

Treinamento                                        Relevância
pepsi                                              0             7
triangular bread                                   0             3
minha buceta gosto pepsi                           0             2
época óculos festa pepsi achando arrasando         0             1
então psicólogo pepsi pessoas pensam sinceramente  0             1
                                                                ..
pepsi gtgtgtgtgt respeito tijucano                 0             1
pepsi kkkkkk                                       0             1
pepsi nossa reles compreensão                      0             1
pepsi pegar largar                                 0             1
                                                   0             1
Length: 132, dtype: int64

In [20]:

#Relevantes
palavras_relevantes = rel['Treinamento']
relev_string = ''
for tweet in palavras_relevantes:
    relev_string += ' ' + tweet
palav_relev = stop_words(tokenization(relev_string))

# Irrelevantes
palavras_irrelevantes = irr['Treinamento']
irrelev_string = ''
for tweet in palavras_irrelevantes:
    irrelev_string += ' ' + tweet
palav_irrelev = stop_words(tokenization(irrelev_string))

Todos os tweets

In [37]:
# pd Series de todas as palavras
todas_palavras = palav_relev + palav_irrelev
serie_todas_palavras = pd.Series(todas_palavras)

# frequencia todas palavras relativa
tabela_todas_palavras_relativa = serie_todas_palavras.value_counts(True)
tabela_todas_palavras_relativa

#frequencia de Todas palavras absolutas
tabela_todas_palavras_abs = serie_todas_palavras.value_counts()


In [22]:
# Palavras sem repetição
palavras_sem_repeticao = list(set(todas_palavras))

In [23]:
# tabela relevantes e irrelevantes
serie_relevantes = pd.Series(palav_relev)
serie_irrelevantes = pd.Series(palav_irrelev)

In [53]:
# frequencias relativas relevantes e irrelevantes
tabela_relevantes_relativa = serie_relevantes.value_counts(True)
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)

rel_freq_abs = serie_relevantes.value_counts()
irr_freq_abs = serie_irrelevantes.value_counts()

## Probabilidades 

In [54]:
 # texto explicando as probabilides que queremos...

In [55]:
#probabilides iniciais

# calculando a probabilidade de ser relevante e irrelente
P_rel = len(palav_relev)/len(todas_palavras)
P_irr = len(palav_irrelev)/len(todas_palavras)

## Naive Bayes

In [56]:
# texto explicando...


## Ingenuidade de Naive Bayes

In [57]:
# texto explicando...


## Suavização de Laplace

In [58]:
# contextualização...

In [63]:
#Suavização de Laplace para corrigir possiveis palavras fora da minha base de dados
def smoothing(palavra, frequencia_absoluta, palavras_sem_repeticao, relevancia, alpha=1):
    resultado = 0
    try:
        absoluta = frequencia_absoluta[palavra]
    except:
        absoluta = 0 #Se a palavra não estiver na lista retorna 0
    if relevancia == 'relevante':
        resultado = (absoluta + alpha)/(len(palav_relev)+alpha*len(palavras_sem_repeticao))
    elif relevancia == 'irrelevante':
        resultado = (absoluta + alpha)/(len(palav_irrelev)+alpha*len(palavras_sem_repeticao))
    return resultado

## Cálculo das Probabilidades condicionais 

In [64]:
# Probabilidade para fazer a desigualdade 
def prob(relevancia,frase):
    if relevancia == 'relevante':
        probTweetdadoR = 1
        for a in frase.split():
            probTweetdadoR *= smoothing(a, rel_freq_abs, palavras_sem_repeticao, relevancia)
        probRdadoTweet = P_rel * probTweetdadoR
        return probRdadoTweet
    elif relevancia == 'irrelevante':
        probTweetdadoI = 1
        for b in frase.split():
            probTweetdadoI *= smoothing(b, irr_freq_abs, palavras_sem_repeticao, relevancia)
        probIdadoTweet = P_irr * probTweetdadoI
        return probIdadoTweet
        
    
    
    

In [65]:
# modelo de naive para comparar as probabilidades
def Naive(tweet):
    if prob('relevante',tweet) > prob('irrelevante',tweet):
        return 1
    else:
        return 0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [66]:
train["Modelo"]=train["Treinamento"].apply(Naive)

In [67]:
train

,Treinamento,Relevância,Modelo
0,namoral refrigerante melhor pepsip existe nenhum,1,1
1,pepsi baseado gravidade sobre pepsi sentido ne...,1,1
2,química kalunga pepsi patrocinadores ficaram b...,1,1
3,mataria refri geladinho pepsi geladeira,1,1
4,dionísio viciado pepsi,1,1
...,...,...,...
295,queria fazer drink laranja beber pepsi mesmo,0,0
296,minha falando pepsi caber geladeira “gordinha”...,0,0
297,triangular bread,0,0
298,acordei enjoo kerelho minha desculpa tomar cop...,1,1


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**